In [ ]:
# Importar las librerías necesarias
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score
)

1. Árbol de Decisiones - CC:SI - ED:NO - Outliers:NO - Balanceo: NO

In [10]:
# --- 1. Carga del Conjunto de Datos ---
data_model_1 = pd.read_csv("../data/subset/clean_subset_lifestyledata_rows5200_seed5200.csv")

# --- 2. Codificación de Etiquetas (Label Encoding) ---
label_encoder = LabelEncoder()
# Se transforma la variable objetivo 'Workout_Type' a valores numéricos.
data_model_1['Workout_Type'] = label_encoder.fit_transform(data_model_1['Workout_Type'])

# --- 3. Codificación One-Hot (One-Hot Encoding) ---
# Se define la lista de columnas categóricas nominales a transformar.
nominal_cols = ['Gender']
# sparse_output=False: Devuelve una matriz densa (array de NumPy) en lugar de una dispersa.
# handle_unknown='ignore': Si aparece una categoría no vista durante la transformación, la ignora.
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# Esto crea nuevas columnas binarias para cada categoría.
encoded = ohe.fit_transform(data_model_1[nominal_cols])
# Se convierte la matriz resultante en un DataFrame con nombres de columna apropiados.
encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(nominal_cols))

# --- 4. Combinación de los Datos Procesados ---
# Se elimina la columna original 'Gender' del DataFrame principal.
# reset_index(drop=True) asegura que los índices se alineen correctamente para la concatenación.
data_model_1 = data_model_1.drop(columns=nominal_cols).reset_index(drop=True)
encoded_df = encoded_df.reset_index(drop=True)

# Se concatenan el DataFrame original y el nuevo DataFrame con las columnas codificadas.
# axis=1 indica que la unión se realiza por columnas.
data_model_1 = pd.concat([data_model_1, encoded_df], axis=1)

# --- 5. Visualización ---
data_model_1.head()

,Age,Weight_kg,Height_m,Max_BPM,Avg_BPM,Resting_BPM,Session_Duration_hours,Calories_Burned,Workout_Type,Gender_Female,Gender_Male
0,21.14,101.05,1.95,171.17,130.81,68.96,0.97,959.43,2,0.0,1.0
1,44.17,41.63,1.78,167.33,158.46,63.95,1.48,1424.35,0,0.0,1.0
2,20.07,63.81,1.78,187.86,137.11,60.93,1.70,1766.64,0,1.0,0.0
3,36.30,59.77,1.78,183.83,120.32,60.01,0.85,1028.50,1,1.0,0.0
4,51.99,57.60,1.56,166.25,151.82,67.97,1.66,1295.80,3,0.0,1.0


In [20]:
# Se definen las características (X) y la variable objetivo (y)
# X contiene todas las columnas EXCEPTO 'Workout_Type'.
X = data_model_1.drop("Workout_Type", axis=1)

# y contiene ÚNICAMENTE la columna 'Workout_Type'.
y = data_model_1["Workout_Type"]

# Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5200)

# Se crea un árbol con el criterio de impureza de Gini y una profundidad máxima de 5
modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=0)
# Se entrena el modelo con los datos de entrenamiento
modelo_gini.fit(X_train, y_train)

# Se crea un árbol con el criterio de Entropía
modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)
# Se entrena el modelo
modelo_entropy.fit(X_train, y_train)

# Se crea un árbol con Entropía y una condición de poda adicional
modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=0)
# Se entrena el modelo
modelo_entropy_pruned.fit(X_train, y_train)

# Predecir con el modelo Gini
y_pred_gini = modelo_gini.predict(X_test)

# Predecir con el modelo de Entropía
y_pred_entropy = modelo_entropy.predict(X_test)

# Predecir con el modelo de Entropía podado
y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test)

# Métricas para el modelo Gini (accuracy, precision, recall, f1-score)
accuracy_gini = accuracy_score(y_test, y_pred_gini)
precision_gini = precision_score(y_test, y_pred_gini, average='weighted')
recall_gini = recall_score(y_test, y_pred_gini, average='weighted')
f1_gini = f1_score(y_test, y_pred_gini, average='weighted')

# Métricas para el modelo de Entropía (accuracy, precision, recall, f1-score)
accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
precision_entropy = precision_score(y_test, y_pred_entropy, average='weighted')
recall_entropy = recall_score(y_test, y_pred_entropy, average='weighted')
f1_entropy = f1_score(y_test, y_pred_entropy, average='weighted')

# Métricas para el modelo de Entropía podado (accuracy, precision, recall, f1-score)
accuracy_entropy_pruned = accuracy_score(y_test, y_pred_entropy_pruned)
precision_entropy_pruned = precision_score(y_test, y_pred_entropy_pruned, average='weighted')
recall_entropy_pruned = recall_score(y_test, y_pred_entropy_pruned, average='weighted')
f1_entropy_pruned = f1_score(y_test, y_pred_entropy_pruned, average='weighted')

# Mostrar las métricas
print("Métricas del Modelo Gini:")
print(f"Accuracy: {accuracy_gini:.4f}, Precision: {precision_gini:.4f}, Recall: {recall_gini:.4f}, F1-Score: {f1_gini:.4f}\n")
print("Métricas del Modelo Entropía:")
print(f"Accuracy: {accuracy_entropy:.4f}, Precision: {precision_entropy:.4f}, Recall: {recall_entropy:.4f}, F1-Score: {f1_entropy:.4f}\n")
print("Métricas del Modelo Entropía Podado:")
print(f"Accuracy: {accuracy_entropy_pruned:.4f}, Precision: {precision_entropy_pruned:.4f}, Recall: {recall_entropy_pruned:.4f}, F1-Score: {f1_entropy_pruned:.4f}\n")



Métricas del Modelo Gini:
Accuracy: 0.7163, Precision: 0.7255, Recall: 0.7163, F1-Score: 0.7171

Métricas del Modelo Entropía:
Accuracy: 0.8077, Precision: 0.8200, Recall: 0.8077, F1-Score: 0.8098

Métricas del Modelo Entropía Podado:
Accuracy: 0.8077, Precision: 0.8200, Recall: 0.8077, F1-Score: 0.8098



## Conclusión:
Se evaluaron tres modelos de árbol de decisión: uno con Gini, otro con entropía y un tercero con entropía podado. Los dos modelos con entropía obtuvieron mejores resultados, con una precisión cercana al 81% y un F1-score de 0.81. Se seleccionó el modelo con entropía podado por ofrecer el mismo nivel de exactitud que el no podado, pero con menor complejidad y mejor capacidad de generalización, logrando un equilibrio óptimo entre rendimiento y simplicidad para predecir el tipo de entrenamiento.